In [35]:
import sqlite3

import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

Create a dataframe:

In [36]:
df = pd.read_sql("SELECT * FROM posts", sqlite3.connect("data.db"))

### Clustering:

In [37]:
posts = df.loc[df["type"] == "POST"]["clean_message"]